In [4]:
!pip install evaluate rouge_score git+https://github.com/google-research/bleurt.git bert_score

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-6ilal083
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-6ilal083
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install openai

In [6]:
!pip install datasets

In [7]:
from datasets import load_dataset
dataset_xlcost = load_dataset("codeparrot/xlcost-text-to-code", "Python-program-level")['test']
dataset_code_x = load_dataset("google/code_x_glue_ct_code_to_text", 'python')['test']
dataset_code_search_net = load_dataset("code-search-net/code_search_net", 'python')['test']

README.md:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

xlcost-text-to-code.py:   0%|          | 0.00/7.61k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/3.92M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/357k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/200k [00:00<?, ?B/s]

...

The repository for code-search-net/code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code-search-net/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] н
The repository for code-search-net/code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code-search-net/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [8]:
def split_text(example):
    example['text'] = example['text'].split('|')[0].strip()
    return example

dataset_xlcost = dataset_xlcost.map(split_text)

dataset_code_x = dataset_code_x.rename_column('docstring', 'text')

dataset_code_search_net = dataset_code_search_net.rename_column('func_documentation_string', 'text')
dataset_code_search_net = dataset_code_search_net.rename_column('func_code_string', 'code')

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

In [49]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

In [96]:
system_message = {
      "role": "system",
      "content": "For each code snippet provided as input, output a very brief summary and nothing else. The summary must not include newline characters or full stop symbols. The number of summary outputs must exactly match the number of input code snippets, with each summary separated by the '$' symbol if there are multiple snippets"
    }

In [102]:
import json

def save_to_json(dataset, summaries, references, left, right, filename):
    output_data = []
    for i in range(len(references)):
        if i >= len(summaries):
            break  # Защита от расхождения длин

        sample_idx = left + i
        entry = {
            "code": dataset[sample_idx]['code'],
            "ideal_answer": references[i],
            "generated_answer": summaries[i].strip()
        }
        output_data.append(entry)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

In [95]:
def get_summarization(model, dataset, left, right):
    codes = ""
    refs = []
    for sample in dataset.select(range(left,right)):
        codes +=sample['code'] + "\n$\n"
        refs.append(sample['text'])
    print(len(codes))
    completion = client.chat.completions.create(
        model = model,
        messages=[
          system_message,
          {
            "role": "user",
            "content": codes
          }
        ]
    )
    summary = completion.choices[0].message.content.split('$')

    return summary, refs

In [12]:
import evaluate
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bleurt = evaluate.load("bleurt", module_type="metric", checkpoint="bleurt-base-128")
bertscore = evaluate.load("bertscore")

In [13]:
def calculate_metrics(preds, refs):
    rouge_result = rouge.compute(predictions=preds, references=refs, rouge_types=["rougeL"])["rougeL"]
    bleu_result = bleu.compute(predictions=preds, references=[[ref] for ref in refs])

    bleurt_scores = bleurt.compute(predictions=preds, references=refs)["scores"]
    avg_bleurt = sum(bleurt_scores) / len(bleurt_scores)

    bert_scores = bertscore.compute(
        predictions=preds,
        references=refs,
        lang="en",
        model_type="microsoft/deberta-large-mnli"
    )
    avg_bert_f1 = sum(bert_scores["f1"]) / len(bert_scores["f1"])


    return rouge_result, bleurt_scores, bert_scores, avg_bert_f1

In [103]:
preds, refs = get_summarization("meta-llama/llama-4-maverick:free", dataset_xlcost, 0, 20)
if len(preds)>len(refs):
    preds = preds[:len(refs)]
print(preds)
save_to_json(dataset_xlcost, preds, refs, 0, 20, "xlcost.json")
rouge_result, bleurt_scores, bert_scores, avg_bert_f1 = calculate_metrics(preds,refs)

print("\rouge_result:", rouge_result)
print("bleurt_scores:", bleurt_scores)
print(f"bert_scores:", bert_scores)
print(f"avg_bert_f1 F1", avg_bert_f1)

12771
['minSum function calculates minimum sum after removing one element from array', '\nmaxAdjacent function calculates maximum adjacent difference for each element in array excluding itself', '\nCountOnes function counts number of ones in a binary representation of a number within a given range', '\nminDivisior function prints minimum divisor and corresponding number for a given input number', "\nLandau_function calculates Landau's function for a given number n", '\ncheckExpression checks if a number can be expressed as sum of prime numbers', "\ncheckArray checks if it's possible to get a subset with k elements having odd count", '\nfunc calculates sum of products of frequencies and corresponding values in array', '\ninsert_element determines number of elements to be inserted in array to make sum equal to twice the XOR', '\ncheckSolution checks if a equals c after performing some operations', '\ncheckSunnyNumber checks if a number is sunny number or not', '\ncountValues counts numbe

In [ ]:
preds, refs = get_summarization("deepseek/deepseek-chat-v3-0324:free", dataset_code_x , 0, 100)
rouge_result, bleurt_scores, bert_scores, avg_bert_f1 = calculate_metrics(preds,refs)

print("\rouge_result:", rouge_result)
print("bleurt_scores:", bleurt_scores)
print(f"bert_scores:", bert_scores)
print(f"avg_bert_f1 F1", avg_bert_f1)


In [ ]:
preds, refs = get_summarization("deepseek/deepseek-chat-v3-0324:free", dataset_code_search_net , 0, 100)
rouge_result, bleurt_scores, bert_scores, avg_bert_f1 = calculate_metrics(preds,refs)

print("\rouge_result:", rouge_result)
print("bleurt_scores:", bleurt_scores)
print(f"bert_scores:", bert_scores)
print(f"avg_bert_f1 F1", avg_bert_f1)
